In [2]:
import pandas as pd
df=pd.read_csv('LoanApprovalPrediction.csv')
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,LP002978,Female,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
594,LP002979,Male,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
595,LP002983,Male,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
596,LP002984,Male,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [3]:
df.duplicated().sum()

0

In [4]:
df.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           12
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           21
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            598 non-null    object 
 1   Gender             598 non-null    object 
 2   Married            598 non-null    object 
 3   Dependents         586 non-null    float64
 4   Education          598 non-null    object 
 5   Self_Employed      598 non-null    object 
 6   ApplicantIncome    598 non-null    int64  
 7   CoapplicantIncome  598 non-null    float64
 8   LoanAmount         577 non-null    float64
 9   Loan_Amount_Term   584 non-null    float64
 10  Credit_History     549 non-null    float64
 11  Property_Area      598 non-null    object 
 12  Loan_Status        598 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 60.9+ KB


In [6]:
# Fill numerical columns with median
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].median(), inplace=True)
df['Dependents'].fillna(df['Loan_Amount_Term'].median(), inplace=True)
# Fill categorical columns with mode
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
df['ApplicantIncome'].astype('int')
df['CoapplicantIncome'].astype('int')
df['Loan_Amount_Term'].astype('int')
df['Credit_History'].astype('int')


0      1
1      1
2      1
3      1
4      1
      ..
593    1
594    1
595    1
596    1
597    0
Name: Credit_History, Length: 598, dtype: int32

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Married'] = le.fit_transform(df['Married'])
df['Education'] = le.fit_transform(df['Education'])
df['Gender'] = le.fit_transform(df['Gender'])
df['Self_Employed'] = le.fit_transform(df['Self_Employed'])
df['Property_Area'] = le.fit_transform(df['Property_Area'])



In [8]:
x = df[[ 'ApplicantIncome','CoapplicantIncome'
       ,'LoanAmount','Credit_History']]
y=df['Loan_Status']

In [9]:
df.drop('Loan_ID', axis=1, inplace=True)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2024)
model = LogisticRegression(max_iter=200)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8166666666666667
              precision    recall  f1-score   support

           N       0.95      0.49      0.65        41
           Y       0.79      0.99      0.88        79

    accuracy                           0.82       120
   macro avg       0.87      0.74      0.76       120
weighted avg       0.84      0.82      0.80       120



In [11]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Feature selection
features = [
    'ApplicantIncome',
    'CoapplicantIncome',
    'LoanAmount',
    'Credit_History'
]

# Column Transformer
feature_select = ColumnTransformer(
    [('num_features', 'passthrough', features)]
)

# Pipeline
pipeline = Pipeline(steps=[
    ('feature_selection', feature_select),
    ('model', LogisticRegression(max_iter=500))
])

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=2024
)

# Cross-validation
cv_scores = cross_val_score(
    pipeline, x_train, y_train, cv=5, scoring='accuracy'
)

print("Cross-validation accuracy scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Fit model
pipeline.fit(x_train, y_train)

# Predictions
train_pred = pipeline.predict(x_train)
test_pred = pipeline.predict(x_test)

# Evaluation
print("Train Accuracy:", accuracy_score(y_train, train_pred))
print("Test Accuracy:", accuracy_score(y_test, test_pred))
print("\nClassification Report:\n", classification_report(y_test, test_pred))


Cross-validation accuracy scores: [0.79166667 0.78125    0.76041667 0.78947368 0.78947368]
Mean CV accuracy: 0.7824561403508772
Train Accuracy: 0.8096234309623431
Test Accuracy: 0.8166666666666667

Classification Report:
               precision    recall  f1-score   support

           N       0.95      0.49      0.65        41
           Y       0.79      0.99      0.88        79

    accuracy                           0.82       120
   macro avg       0.87      0.74      0.76       120
weighted avg       0.84      0.82      0.80       120



In [15]:
import pickle

# Save the trained pipeline
with open("loan_approval_model.pkl", "wb") as file:
    pickle.dump(pipeline, file)

print("Model saved successfully as loan_approval_model.pkl")

Model saved successfully as loan_approval_model.pkl


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# إنشاء النموذج
rf = RandomForestClassifier(
    n_estimators=200,     
    max_depth=None,      
    random_state=42
)

# تدريب النموذج
rf.fit(x_train, y_train)

# التنبؤ
y_pred = rf.predict(x_test)

# تقييم النموذج
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8

Classification Report:
               precision    recall  f1-score   support

           N       0.76      0.61      0.68        41
           Y       0.82      0.90      0.86        79

    accuracy                           0.80       120
   macro avg       0.79      0.75      0.77       120
weighted avg       0.80      0.80      0.79       120


Confusion Matrix:
 [[25 16]
 [ 8 71]]


In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(
    criterion='gini',
    max_depth=None,
    random_state=42
)

scores = cross_val_score(dt, x, y, cv=5)
print("CV accuracy:", scores.mean())


CV accuracy: 0.7174089635854342


In [14]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x, y, cv=5)
print("CV accuracy:", scores.mean())
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,       # more trees
    max_depth=7,            # limit depth
    min_samples_split=10,   # avoid overfitting
    min_samples_leaf=5,
    random_state=42
)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

# تقييم النموذج
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


CV accuracy: 0.7793137254901961
Accuracy: 0.8166666666666667

Classification Report:
               precision    recall  f1-score   support

           N       0.95      0.49      0.65        41
           Y       0.79      0.99      0.88        79

    accuracy                           0.82       120
   macro avg       0.87      0.74      0.76       120
weighted avg       0.84      0.82      0.80       120


Confusion Matrix:
 [[20 21]
 [ 1 78]]
